<a href="https://colab.research.google.com/github/poseidon2022/Retreival-Augumented-Generation/blob/main/RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
!pip install --upgrade fsspec==2024.6.1
!pip install -qU \
  langchain==0.0.300 \
  datasets==2.14.6 \
  pinecone-client==2.2.4 \
  tiktoken==0.5.1
!pip install langchain_google_genai

  Using cached fsspec-2024.6.1-py3-none-any.whl.metadata (11 kB)
Using cached fsspec-2024.6.1-py3-none-any.whl (177 kB)
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2023.10.0
    Uninstalling fsspec-2023.10.0:
      Successfully uninstalled fsspec-2023.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datasets 2.14.6 requires fsspec[http]<=2023.10.0,>=2023.1.0, but you have fsspec 2024.6.1 which is incompatible.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.6.1 requires fsspec==2024.6.1, but you have fsspec 2023.10.0 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 396.2/396.2 kB 21.6 MB/s eta 0:00:

In [11]:
from google.colab import userdata
GOOGLE_API_KEY = userdata.get('GOOGLE_API_KE')


In [14]:
from langchain_google_genai import ChatGoogleGenerativeAI
llm = ChatGoogleGenerativeAI(model="gemini-1.5-pro",
                             google_api_key = GOOGLE_API_KEY)

#Example generation
llm.invoke("Write me a ballad about LangChain").content

"(Verse 1)\nIn a realm of data, vast and wide,\nWhere information flows like a tide,\nThere arose a tool, both strong and bold,\nLangChain its name, a story to be told.\n\n(Verse 2)\nWith chains of logic, it weaves its spell,\nConnecting modules, stories to tell,\nFrom language models, it draws its might,\nUnlocking insights, day and night.\n\n(Verse 3)\nChatbots it crafts, with voices clear,\nAnswering queries, dispelling fear,\nDocuments summarized, knowledge refined,\nA treasure trove of wisdom, you'll find.\n\n(Verse 4)\nGenerative tales, it spins with grace,\nCode it can write, at a rapid pace,\nFrom poems to scripts, its talents abound,\nA creative powerhouse, profound.\n\n(Verse 5)\nDevelopers gather, from far and wide,\nTo harness its power, with eager stride,\nWith every project, its legend grows,\nLangChain, the future, everyone knows.\n\n(Verse 6)\nSo let us sing of this wondrous tool,\nA testament to innovation's rule,\nLangChain, the chain that sets us free,\nTo explore ne

In [36]:
from langchain.schema import (
    SystemMessage,
    HumanMessage,
    AIMessage
)

#let us have a continuous example
messages = [
    ("system", "You are a helpful assistant."),
    ("human", "Hi AI how are you today?"),
    ("ai", "I am great, how can I help you today?"),
    ("human", "I want to know more about string theory.")
]

In [37]:
res = llm.invoke(messages)
res.content

'That\'s a fascinating topic! String theory is a complex and mind-bending area of theoretical physics. To give you the best understanding, I\'ll break it down into digestible parts:\n\n**The Basics**\n\n* **Imagine...tiny, vibrating strings:** At its core, string theory proposes that the fundamental building blocks of the universe are not point-like particles (like electrons or quarks), but rather incredibly tiny, one-dimensional objects called "strings."\n* **Vibrations create reality:**  Just like a guitar string vibrating at different frequencies produces different musical notes, these strings vibrate in different ways, and these vibrations are what we perceive as different particles and forces.\n* **Unifying forces:** One of the main goals of string theory is to provide a single framework that can explain all four fundamental forces of nature: gravity, electromagnetism, the strong nuclear force, and the weak nuclear force.\n\n**Key Concepts**\n\n* **Extra Dimensions:**  String theo

In [38]:
messages.append(("ai", res.content))
messages

[('system', 'You are a helpful assistant.'),
 ('human', 'Hi AI how are you today?'),
 ('ai', 'I am great, how can I help you today?'),
 ('human', 'I want to know more about string theory.'),
 ('ai',
  'That\'s a fascinating topic! String theory is a complex and mind-bending area of theoretical physics. To give you the best understanding, I\'ll break it down into digestible parts:\n\n**The Basics**\n\n* **Imagine...tiny, vibrating strings:** At its core, string theory proposes that the fundamental building blocks of the universe are not point-like particles (like electrons or quarks), but rather incredibly tiny, one-dimensional objects called "strings."\n* **Vibrations create reality:**  Just like a guitar string vibrating at different frequencies produces different musical notes, these strings vibrate in different ways, and these vibrations are what we perceive as different particles and forces.\n* **Unifying forces:** One of the main goals of string theory is to provide a single frame

In [39]:
messages.append(("human", "tell me more"))
llm.invoke(messages).content

'Okay, you want to dive deeper! Here are some more intriguing aspects of string theory we can explore:\n\n**1. The Landscape Problem and the Multiverse:**\n\n* One prediction arising from string theory is the existence of a vast "landscape" of possible universes, each with its own set of physical laws and constants. This is because the extra dimensions in string theory can fold up in a mind-boggling number of ways (estimated to be around 10^500!), each configuration leading to a different universe.\n* This idea, while mind-blowing, presents a challenge. If there are so many possible universes, why do we live in this particular one with these specific laws of physics? This is an active area of research and debate within the field.\n\n**2. D-Branes: Where Strings Get Stuck**\n\n* String theory also introduces the concept of "D-branes," which are multi-dimensional objects (membranes) that exist within the higher-dimensional space. \n* Strings can be thought of as being "attached" to these